In [3]:
# Common 
import os
import keras
import numpy as np 
import tensorflow as tf

# Data
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization 
import plotly.express as px

# Models
from keras import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101V2

In [4]:
info_file = './monkeypox-skin-lesion-dataset/Monkeypox_Dataset_metadata.csv'
info = pd.read_csv(info_file)
info.head()

,ImageID,Label
0,M01_01,Monkeypox
1,M01_02,Monkeypox
2,M01_03,Monkeypox
3,M01_04,Monkeypox
4,M02_01,Monkeypox


In [5]:
fig = px.pie(
    names=info.Label.unique(),
    values=info.Label.value_counts(),
    hole=0.2,
    width=500,
    height=500
)
fig.update_layout({'title':{'text':'Class Distribution','x':0.45}})
fig.show()

In [6]:
# Data Generators
train_gen = ImageDataGenerator(rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

# Loading Data
train_data = train_gen.flow_from_directory("./monkeypox-skin-lesion-dataset/Augmented Images/Augmented Images",target_size=(256,256),shuffle=True,class_mode='binary')
valid_data = valid_gen.flow_from_directory('./monkeypox-skin-lesion-dataset/Augmented Images/Augmented Images',target_size=(256,256),shuffle=True,subset='training',class_mode='binary')
test_data = valid_gen.flow_from_directory('./monkeypox-skin-lesion-dataset/Original Images/Original Images',target_size=(256,256),shuffle=True,subset='validation',class_mode='binary')

Found 3192 images belonging to 2 classes.
Found 2555 images belonging to 2 classes.
Found 45 images belonging to 2 classes.


In [8]:
with tf.device('/GPU:0'):
  base_model = ResNet101V2(
      include_top=False,
      input_shape=(256,256,3)
  )
  base_model.trainable = False

  model = Sequential([
      base_model,
      GlobalAveragePooling2D(),
      Dense(256,activation='relu'),
      BatchNormalization(),
      Dense(164,activation='relu'),
      BatchNormalization(),
      Dense(1,activation='sigmoid')
  ])

  model.compile(
      loss='binary_crossentropy',
      optimizer='Adam',
      metrics=['accuracy']
  )

  cb = [EarlyStopping(patience=5,monitor='val_accuracy',mode='max',restore_best_weights=True),ModelCheckpoint("ResNet101V2-01.h5",save_best_only=True)]

  model.fit(
      train_data,
      epochs=50,
      validation_data=valid_data,
      callbacks=cb
  )


Epoch 1/50
100/100 [==============================] - 57s 437ms/step - loss: 0.2403 - accuracy: 0.8994 - val_loss: 0.1122 - val_accuracy: 0.9601
Epoch 2/50
100/100 [==============================] - 38s 375ms/step - loss: 0.0803 - accuracy: 0.9727 - val_loss: 0.0495 - val_accuracy: 0.9879
Epoch 3/50
100/100 [==============================] - 38s 376ms/step - loss: 0.0561 - accuracy: 0.9806 - val_loss: 0.0335 - val_accuracy: 0.9902
Epoch 4/50
100/100 [==============================] - 35s 347ms/step - loss: 0.0458 - accuracy: 0.9865 - val_loss: 0.0379 - val_accuracy: 0.9914
Epoch 5/50
100/100 [==============================] - 37s 374ms/step - loss: 0.0411 - accuracy: 0.9865 - val_loss: 0.0192 - val_accuracy: 0.9945
Epoch 6/50
100/100 [==============================] - 38s 382ms/step - loss: 0.0323 - accuracy: 0.9890 - val_loss: 0.0134 - val_accuracy: 0.9980
Epoch 7/50
100/100 [==============================] - 35s 345ms/step - loss: 0.0459 - accuracy: 0.9828 - val_loss: 0.0312 - val_ac

In [7]:
model = keras.models.load_model('.\ResNet101V2-01.h5') # Training Accuracy : 99.0% Validation Accuracy : 100.0%
model.evaluate(test_data)

2/2 [==============================] - 30s 4s/step


array([[9.9999583e-01],
       [9.9999630e-01],
       [7.6163774e-06],
       [9.1677402e-06],
       [9.9997699e-01],
       [9.9997509e-01],
       [1.3861526e-04],
       [9.9998355e-01],
       [9.9996912e-01],
       [1.0000000e+00],
       [2.2852828e-05],
       [9.9999785e-01],
       [9.9998248e-01],
       [8.3146122e-05],
       [1.8274810e-06],
       [9.9994481e-01],
       [9.9905282e-01],
       [5.7386592e-06],
       [2.2426688e-05],
       [5.6562232e-07],
       [1.2229463e-04],
       [9.9999988e-01],
       [9.9998522e-01],
       [2.2115833e-04],
       [9.9999714e-01],
       [9.2879764e-06],
       [9.9999988e-01],
       [9.9999714e-01],
       [9.9995208e-01],
       [1.2940054e-06],
       [6.0580004e-07],
       [9.9815649e-01],
       [9.9953628e-01],
       [4.3020733e-05],
       [9.9998283e-01],
       [7.4648859e-05],
       [9.9999988e-01],
       [6.8752666e-07],
       [9.9999952e-01],
       [9.9999845e-01],
       [5.4459488e-06],
       [9.999786

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 8, 8, 2048)        42626560  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 164)               42148     
                                                                 
 batch_normalization_1 (Batc  (None, 164)              6

In [11]:
from sklearn.model_selection import KFold

n_split=4

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test=X[train_data],X[train_data]
  y_train,y_test=Y[train_index],Y[test_index]
  
  model.compile(
      loss='binary_crossentropy',
      optimizer='Adam',
      metrics=['accuracy']
  )
  
  model.fit(
      train_data,
      epochs=50,
      validation_data=valid_data,
      callbacks=cb
  )
  
  print('Model evaluation ',model.evaluate(x_test,y_test))

NameError: name 'X' is not defined